# Spring's constant measuring

## Importing libraries

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import os
from functools import reduce

from modules import LinearRegression

## Load data

The following code creates a table (dict) with entries that have:
- as key, the name of the file from which the data is read
- as value, the DataFrame returned by pandas when reading that file

Getting data from this table involves using the file from which the data is read as key and then parsing the columns of the corresponding DataFrame.

e.g.:

```
masses_measurements = datasets['misure_masse']
used_objects = masses_measurements[masses_measurements.columns[0]]
masses = masses_measurements[masses_measurements.columns[1]]
```

In [11]:
# collect filenames of exported data

data_dir = "./data"

csv_data = []
excel_data = []

for root, dirs, files in os.walk(data_dir):
    for file in files:
        current_file_path = os.path.join(root, file).replace(
            "\\", "/"
        )  # fix the unbearably frustrating flaws of the wanna-be OS... "Windows"
        if "xlsx" in file:
            excel_data.append(current_file_path)
        elif "csv" in file:
            csv_data.append(current_file_path)
    break  # stop at first recursion level: only ./data

print(f"CSV:\n{csv_data}\nEXCEL:\n{excel_data}")

CSV:
['./data/sonar_non-pretensionata_statico.csv', './data/sonar_pretensionata_statico.csv', './data/sonar_non-pretensionata_dinamico.csv', './data/sonar_pretensionata_dinamico.csv']
EXCEL:
['./data/masse_sonar_non-pretensionata.xlsx', './data/calibro_pretensionata_statico.xlsx', './data/masse_sonar_pretensionata.xlsx', './data/misure_masse.xlsx', './data/calibro_non-pretensionata_statico.xlsx']


In [12]:
def read_csv(filename: str):
    data = pd.read_csv(filename, sep=";").replace(",", ".", regex=True)

    data.dropna(inplace=True)
    data.drop(
        index=data.index[0], axis=0, inplace=True
    )  # instrument error causes first value to be nonsensical

    for col in data.columns:
        data[col] = data[col].apply(pd.to_numeric)

    return data


def read_excel(filename: str):
    data = pd.read_excel(filename)
    return data


datasets = dict()

for file in csv_data + excel_data:
    key = os.path.basename(file).split(".")[0]
    if "csv" in file:
        datasets[key] = read_csv(file)
    elif "xlsx" in file:
        datasets[key] = read_excel(file)

print(datasets.keys())

for k in datasets.keys():
    print(f"\nkey: {k}")
    datasets[k].info()

dict_keys(['sonar_non-pretensionata_statico', 'sonar_pretensionata_statico', 'sonar_non-pretensionata_dinamico', 'sonar_pretensionata_dinamico', 'masse_sonar_non-pretensionata', 'calibro_pretensionata_statico', 'masse_sonar_pretensionata', 'misure_masse', 'calibro_non-pretensionata_statico'])

key: sonar_non-pretensionata_statico
<class 'pandas.core.frame.DataFrame'>
Index: 1027 entries, 1 to 1027
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Position (mm) Run #1  1027 non-null   float64
 1   Position (mm) Run #2  1027 non-null   float64
 2   Position (mm) Run #3  1027 non-null   float64
 3   Position (mm) Run #4  1027 non-null   float64
 4   Position (mm) Run #5  1027 non-null   float64
 5   Position (mm) Run #6  1027 non-null   float64
dtypes: float64(6)
memory usage: 56.2 KB

key: sonar_pretensionata_statico
<class 'pandas.core.frame.DataFrame'>
Index: 1029 entries, 1 to 1029
Data columns (t

## Analyze data

For the **static method**, the sonar's signal is hypothesized to have a very small amplitude, so the measure of the mass' displacement is the mean of the recorded positions.<BR><BR>
For the **dynamic method**, we isolate the peaks of the oscillating signal and compute the mean of the period between two adjacent peaks. From there, the resulting period is used to approximate the frequency of the oscillation.

In [35]:
static_keys = list(filter(lambda k: "statico" in k, datasets.keys()))
dynamic_keys = list(filter(lambda k: "dinamico" in k, datasets.keys()))

print(f"{static_keys}\n{dynamic_keys}")

static_datasets = {k: datasets[k] for k in static_keys}
dynamic_datasets = {k: datasets[k] for k in dynamic_keys}

masses = datasets["misure_masse"]  # it's going to be used a lot, so...

masse_crescenti = np.zeros(10)
masse_crescenti[0] = masses["massa [g]"][2] + masses["massa [g]"][3]
for i in range(0, 9):
    masse_crescenti[i + 1] = masse_crescenti[i] + masses["massa [g]"][i + 4]

['sonar_non-pretensionata_statico', 'sonar_pretensionata_statico', 'calibro_pretensionata_statico', 'calibro_non-pretensionata_statico']
['sonar_non-pretensionata_dinamico', 'sonar_pretensionata_dinamico']
[ 28.15  47.69  67.32  87.18 107.31 127.3  146.96 166.87 186.86 206.87]


### Static method

In [17]:
# Analyze static method

averages = np.zeros(7)
for run in range(7):
    averages[run] += np.mean(
        datasets["sonar_pretensionata_statico"][f"Position (mm) Run #{run+1}"]
    )

averages = np.round_(averages, decimals=2)
print(averages)

k = 9.81 * (masses[-1] - masses[0]) / (averages[-1] - averages[0])
print(k)

[  0.19 -12.71 -25.71 -38.34 -51.45 -64.31 -77.2 ]


KeyError: -1

### Dynamic method

In [ ]:
# Analyze dynamic method


def discrete_derivative(domain: list, curve: list):
    d = [0]
    for i in range(1, len(curve)):
        d.append((curve[i] - curve[i - 1]) / (domain[i] - domain[i - 1]))
    return d


# windowed peaks isolation
def windowed_peaks(signal: list):
    peaks = []
    for i in range(2, len(signal) - 2, 4):
        if (
            reduce(
                lambda x, y: x * y,
                discrete_derivative(list(range(i - 2, i + 3)), signal[i - 2 : i + 3]),
            )
            < 0
        ):
            peaks.append(max(signal[i - 2 : i + 3]))
    return peaks

In [ ]:
def calculate_mean_period(column_id: str):
    wf = data[column_id].array
    for v in wf:
        print(windowed_peaks(wf))


# calculate_mean_period(data[data.columns[0]])

# for col in data.columns:
# calculate_mean_period(col)